<a href="https://colab.research.google.com/github/adheeb4/MLNN-Project-Group-20/blob/main/Group_Project_Group_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
from PIL import Image
from google.colab import drive
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras import layers, models, optimizers, losses, callbacks,\
                             regularizers
from keras.activations import relu, sigmoid
from keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
train_directory = "/content/gdrive/MyDrive/Group_Project_Data 2/Group_Project_Data/Train"
validation_directory = "/content/gdrive/MyDrive/Group_Project_Data 2/Group_Project_Data/Valid"


In [ ]:
train_data_gen = ImageDataGenerator(rescale = 1./255, shear_range =0.2, zoom_range = 0.2, horizontal_flip = True)
val_data_gen = ImageDataGenerator(rescale = 1./255)

In [ ]:
train_gen = train_data_gen.flow_from_directory(
    train_directory,
    target_size = (64, 64),
    batch_size = 64,
    class_mode = "binary",
    classes =["Real", "Fake"],
    shuffle=True,
    seed=42,
    subset = "training"
)

Found 6000 images belonging to 2 classes.


In [ ]:
val_gen = val_data_gen.flow_from_directory(
    validation_directory,
    target_size = (64, 64),
    batch_size = 64,
    class_mode = "binary",
    classes =["Real", "Fake"],
    shuffle=False,
    seed=42,
)

Found 2000 images belonging to 2 classes.


In [ ]:
input_layer = Input((64, 64, 1), name="Input")

conv_1 = Conv2D(32, (3, 3), activation=relu, padding="same", name="conv_1")(input_layer)
conv_2 = Conv2D(32, (3, 3), activation=relu, padding="same", name="conv_2")(conv_1)
pool_1 = MaxPooling2D(pool_size=(2, 2),name="pool_1")(conv_2)
dropout1 = Dropout(0.2, name="dropout_1")(pool_1)

conv_3 = Conv2D(64, (3, 3), activation=relu, padding="same", name="conv_3")(dropout1)
conv_4 = Conv2D(64, (3, 3), activation=relu, padding="same", name="conv_4")(conv_3)
pool_2 = MaxPooling2D(pool_size=(2, 2),name="pool_2")(conv_4)
dropout_2 = Dropout(0.3, name="dropout_2")(pool_2)

conv_5 = Conv2D(128, (3, 3), activation=relu, padding="same", name="conv_5")(dropout_2)
conv_6 = Conv2D(128, (3, 3), activation=relu, padding="same", name="conv_6")(conv_5)
pool_3 = MaxPooling2D(pool_size=(2, 2),name="pool_3")(conv_6)
dropout_3 = Dropout(0.4, name="dropout_3")(pool_2)

flat = Flatten(name="flat")(pool_3)
fc_1 = Dense(units=512, activation=relu, name="fc_1")(flat)
dropout_4 = Dropout(0.5, name="dropout_4")(fc_1)

Output = Dense(units=1, activation=sigmoid, name="Output")(fc_1)

model = Model(inputs=input_layer, outputs=Output, name="Galaxies")
model.summary()

Model: "Galaxies"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 256, 256, 3)]     0         
                                                                 
 conv_1 (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 conv_2 (Conv2D)             (None, 256, 256, 32)      9248      
                                                                 
 pool_1 (MaxPooling2D)       (None, 128, 128, 32)      0         
                                                                 
 conv_3 (Conv2D)             (None, 128, 128, 64)      18496     
                                                                 
 conv_4 (Conv2D)             (None, 128, 128, 64)      36928     
                                                                 
 pool_2 (MaxPooling2D)       (None, 64, 64, 64)        0  

In [ ]:
Early_Stop = callbacks.EarlyStopping(monitor='val_loss', patience=3)

model.compile(optimizer=optimizers.Adam(), loss=losses.BinaryCrossentropy(), metrics=["accuracy"], callbacks=[Early_Stop])

In [ ]:

Early_Stop = callbacks.EarlyStopping(monitor='val_loss', patience=3)
optimizers=[SGD, RMSprop, Nadam]
learning_rates=[0.01, 0.001, 0.0001]
histories = [[None]*len(learning_rates) for _ in range(len(optimizers))]

for i, optimizer in enumerate(optimizers):
    for j, lr in enumerate(learning_rates):
        print("Optimizer: "+str(optimizer.__name__)+", Learning Rate:"+
              str(learning_rates[j]))
        reinitialize(cifar_model)
        cifar_model.compile(optimizer=optimizer(lr), 
                            loss=losses.SparseCategoricalCrossentropy(),
                            metrics=["accuracy"])
        history = cifar_model.fit(x=x_train, y=y_train, epochs=52, 
                                  validation_data=(x_test, y_test), 
                                  batch_size=128, callbacks=[Early_Stop])
        histories[i][j]=history

In [ ]:
history = model.fit(train_gen,
                    steps_per_epoch=train_gen.samples // train_gen.batch_size,
                    epochs=10,
                    validation_data=val_gen,
                    validation_steps=val_gen.samples // val_gen.batch_size)

test
test1

93/93 [==============================] - 144s 1s/step - loss: 0.1863 - accuracy: 0.9153 - val_loss: 0.0150 - val_accuracy: 0.9975
Epoch 2/10
93/93 [==============================] - 116s 1s/step - loss: 0.0425 - accuracy: 0.9855 - val_loss: 0.0132 - val_accuracy: 0.9950
Epoch 3/10
93/93 [==============================] - 119s 1s/step - loss: 0.0115 - accuracy: 0.9968 - val_loss: 0.0033 - val_accuracy: 0.9995
Epoch 4/10
93/93 [==============================] - 116s 1s/step - loss: 0.0486 - accuracy: 0.9877 - val_loss: 0.0137 - val_accuracy: 0.9955
Epoch 5/10
93/93 [==============================] - 115s 1s/step - loss: 0.0166 - accuracy: 0.9938 - val_loss: 0.0108 - val_accuracy: 0.9970
Epoch 6/10
93/93 [==============================] - 115s 1s/step - loss: 0.0053 - accuracy: 0.9975 - val_loss: 0.0052 - val_accuracy: 0.9990
Epoch 7/10
93/93 [==============================] - 116s 1s/step - loss: 0.0037 - accuracy: 0.9995 - val_loss: 7.7639e-04 - val_accuracy: 1.0000
Epoch 8/10
93/93 [==

In [ ]:
for i in range(len(optimizers)):
  for j in range(len(learning_rates)):

    plt.figure()
    plt.plot(histories[i][j].history["loss"], label="Training")
    plt.plot(histories[i][j].history["val_loss"], label="Validation")
    plt.xlabel("Number of Epochs")
    plt.ylabel("Loss")
    plt.legend(loc=0)
    plt.title("Model Loss\n"+"Optimizer: "+str(optimizers[i].__name__)+
              ", Learning Rate: "+str(learning_rates[j]))

    plt.figure()
    plt.plot(histories[i][j].history["accuracy"], label="Training")
    plt.plot(histories[i][j].history["val_accuracy"], label="Validation")
    plt.xlabel("Number of Epochs")
    plt.ylabel("Accuracy")
    plt.legend(loc=0)
    plt.title("Model Accuracy\n"+"Optimizer: "+str(optimizers[i].__name__)+
              ", Learning Rate: "+str(learning_rates[j]))


    plt.show()
